In [1]:
''' This is a sample for celestial navigation for a stationary observer 
    © August Linnman, 2025, email: august@linnman.net
    MIT License (see LICENSE file)

    Jupyter notebook for ordinary three-fix celestial navigation (stationary).
'''

# Importing Python libraries
from time import time
from starfix import Sight, SightCollection, get_representation, get_google_map_string,\
                    IntersectError, LatLonGeodetic, Circle

In [2]:
# SIGHTS

def get_starfixes (drp_pos : LatLonGeodetic) -> SightCollection :
    ''' Returns a list of used star fixes (SightCollection) '''

    a = Sight ( object_name          = "Capella",
                set_time             = "2024-09-17 23:36:13+00:00",
                measured_alt         = "33 :9    :34",
                estimated_position   = drp_pos
                )

    b = Sight ( object_name          = "Moon",
                set_time             = "2024-09-17 23:41:13+00:00",
                measured_alt         = "48 :22  :5.2"
                )

    c = Sight ( object_name          = "Vega",
                set_time             = "2024-09-17 23:46:13+00:00",
                measured_alt         = "25 :39:4"
                )
    return SightCollection ([a, b, c])


In [ ]:
# SIGHT REDUCTION.

the_pos = LatLonGeodetic (35, -10) # Rough DRP position

intersections = None
collection = None
the_map = None
try:
    intersections, _, _, collection =\
            SightCollection.get_intersections_conv (return_geodetic=True,
                                                    estimated_position=the_pos,
                                                    get_starfixes=get_starfixes)
    assert intersections is not None
    assert collection is not None
    print (get_representation(intersections,1))
    assert isinstance (intersections, LatLonGeodetic)
    print ("Google Map Coordinate = " + get_google_map_string(intersections,4)) 

except IntersectError as ve:
    print ("Cannot perform a sight reduction. Bad sight data.\n" + str(ve))
    if ve.coll_object is not None:
        if isinstance (ve.coll_object, SightCollection):
            collection = ve.coll_object

if isinstance (intersections, tuple):
    intersections = None

if collection is not None:
    the_map = collection.render_folium (intersections)
the_map
